In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
import tensorflow as tf

In [6]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical

In [7]:
# Step 1: Load and preprocess images
data_dir = 'dataset_blood_group'
img_size = (128, 128)


In [8]:
X = []
y = []

In [9]:
for label in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, label)
    if not os.path.isdir(folder_path):
        continue

    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        try:
            img = load_img(img_path, target_size=img_size, color_mode='grayscale')
            img_array = img_to_array(img) / 255.0
            X.append(img_array)
            y.append(label)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")

In [10]:
X = np.array(X)

In [11]:
y = np.array(y)

In [12]:
# Step 2: Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

In [13]:
# Step 3: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

In [14]:
# Step 4: Build CNN model
from tensorflow.keras import layers, models, Input

model = models.Sequential([
    Input(shape=(128, 128, 1)),  # Define input shape explicitly
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(8, activation='softmax')  # 8 classes for blood groups
])


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Step 5: Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 35s 222ms/step - accuracy: 0.3090 - loss: 2.0198 - val_accuracy: 0.7707 - val_loss: 0.6192
Epoch 2/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 32s 217ms/step - accuracy: 0.8157 - loss: 0.5205 - val_accuracy: 0.8517 - val_loss: 0.3842
Epoch 3/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 33s 221ms/step - accuracy: 0.8570 - loss: 0.3722 - val_accuracy: 0.8636 - val_loss: 0.3481
Epoch 4/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 32s 220ms/step - accuracy: 0.8949 - loss: 0.2790 - val_accuracy: 0.8738 - val_loss: 0.2873
Epoch 5/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 32s 217ms/step - accuracy: 0.9129 - loss: 0.2270 - val_accuracy: 0.8372 - val_loss: 0.4200
Epoch 6/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 32s 218ms/step - accuracy: 0.9244 - loss: 0.1990 - val_accuracy: 0.8585 - val_loss: 0.3535
Epoch 7/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 33s 224ms/step - accuracy: 0.9580 - loss: 0.1203 - val_accuracy: 0.8875 - val_loss: 0.2659
Epoch 8/10
147/147 ━━━━━━━━━━━━━━━━━━━━ 32s 219ms/step - accuracy: 0.9820 - loss: 0

In [17]:
# Step 6: Evaluate
y_pred = model.predict(X_test)

37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step


In [18]:
y_pred_classes = np.argmax(y_pred, axis=1)

In [19]:
y_true = np.argmax(y_test, axis=1)

In [20]:
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=le.classes_))

Classification Report:
              precision    recall  f1-score   support

          A+       0.99      0.89      0.94       129
          A-       0.93      0.89      0.91       176
         AB+       0.83      0.95      0.88       132
         AB-       0.90      0.87      0.89       158
          B+       0.89      0.96      0.92       121
          B-       0.90      0.94      0.92       156
          O+       0.92      0.78      0.85       162
          O-       0.82      0.91      0.86       139

    accuracy                           0.90      1173
   macro avg       0.90      0.90      0.90      1173
weighted avg       0.90      0.90      0.90      1173



In [21]:
model.save("blood_group_model.keras")
print("Model saved as 'blood_group_model.keras'")


Model saved as 'blood_group_model.keras'


In [22]:
import os
print(os.getcwd())


C:\Users\User\Desktop\blood group detect useing fingerprint


In [23]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the trained model
model = load_model("blood_group_model.keras")

# Set the full path to your fingerprint image
img_path = r"C:\Users\User\Desktop\blood group detect useing fingerprint\dataset_blood_group\AB+\cluster_4_4.BMP"  

# Load and preprocess the image
img = load_img(img_path, target_size=(128, 128), color_mode='grayscale')
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make prediction
prediction = model.predict(img_array)
predicted_index = np.argmax(prediction)

# Optional: Convert class index to blood group label
blood_groups = ['A+', 'A−', 'AB+', 'AB−', 'B+', 'B−', 'O+', 'O−'] 
predicted_label = blood_groups[predicted_index]

print(f"Predicted Blood Group: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
Predicted Blood Group: AB+
